### ドライブのマウント

In [1]:
# Googleドライブをマウント
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### ライブラリ・モジュールのインポート

In [2]:
# ライブラリの準備
!pip install timm
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn.functional as F
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from timm.scheduler import CosineLRScheduler
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import math
import copy
import random
from PIL import Image
import pickle

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 22.8 MB/s eta 0:00:00


### シード値の設定

In [3]:
# シード値を設定
def fix_seed(seed=1234):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

fix_seed(seed=1234)

### データセットの準備

In [4]:
# Colabのローカルファイルに自作データをコピー（初めからローカルファイルにアップロードする場合は不要）
!cp -r "/content/drive/MyDrive/HandSigns/" "/content/HandSigns/"

In [5]:
# 自作データをリサイズして、ファイル構造を保ったまま新しいフォルダに保存する関数を定義
def resize_images(base_dir, new_base_dir, image_size=(32, 32)):
    for folder in os.listdir(base_dir):
        folder_path = os.path.join(base_dir, folder)
        new_folder_path = os.path.join(new_base_dir, folder)
        # リサイズした自作データを保存する新しいフォルダを作成
        os.makedirs(new_folder_path, exist_ok=True)
        for filename in os.listdir(folder_path):
            file_path = os.path.join(folder_path, filename)
            # 拡張子の確認
            if file_path.endswith(('.jpg', '.jpeg')):
                # 画像を展開
                img = Image.open(file_path)
                # 画像をリサイズ
                img_resized = img.resize(image_size)
                # リサイズした画像を新しいフォルダに保存
                img_resized.save(os.path.join(new_folder_path, filename))

# 学習用の自作データのディレクトリを設定
train_base_dir = '/content/HandSigns/train'
# テスト用の自作データのディレクトリを設定
test_base_dir = '/content/HandSigns/test'

# リサイズした学習用の自作データを保存するディレクトリを設定
new_train_base_dir = '/content/HandSigns/resized_train_32'
# リサイズしたテスト用の自作データを保存するディレクトリを設定
new_test_base_dir = '/content/HandSigns/resized_test_32'
# リサイズする大きさを設定
image_size = (32, 32)

# 学習用の自作データをリサイズ
resize_images(train_base_dir, new_train_base_dir, image_size)
# テスト用の自作データをリサイズ
resize_images(test_base_dir, new_test_base_dir, image_size)

In [6]:
# 平均値と標準偏差を計算するためのHandSignsの前処理を定義
pre_transform_handsigns = transforms.Compose([
    transforms.ToTensor()
])

# 平均値と標準偏差を計算するためのHandSignsデータセットの前処理を定義
pre_train_dataset_handsigns = torchvision.datasets.ImageFolder("/content/HandSigns/resized_train_32", transform=pre_transform_handsigns)

# 平均値と標準偏差を計算するための変数を初期化
pre_mean_handsigns = 0.0
pre_std_handsigns = 0.0
pre_total_samples_handsigns = len(pre_train_dataset_handsigns)

# データセットのすべてのデータポイントに対して平均値と標準偏差を計算
for data in pre_train_dataset_handsigns:
    pre_image, _ = data
    pre_mean_handsigns += pre_image.mean(dim=(1, 2))  # テンソルのチャンネルごとに平均を計算
    pre_std_handsigns += pre_image.std(dim=(1, 2))    # テンソルのチャンネルごとに標準偏差を計算

# データセット全体の平均値と標準偏差を計算
pre_mean_handsigns /= pre_total_samples_handsigns
pre_std_handsigns /= pre_total_samples_handsigns

print("データセット全体の平均値: ", pre_mean_handsigns)
print("データセット全体の標準偏差: ", pre_std_handsigns)

データセット全体の平均値:  tensor([0.6954, 0.6754, 0.6450])
データセット全体の標準偏差:  tensor([0.1331, 0.1565, 0.1689])


In [7]:
# 学習用のHandSignsデータセットの前処理を定義
train_transform_handsigns = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=pre_mean_handsigns, std=pre_std_handsigns)
    ])

# テスト用のHandSignsデータセットの前処理を定義
test_transform_handsigns = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=pre_mean_handsigns, std=pre_std_handsigns)
    ])

# 学習用のHandSignsデータセットの読み込み
train_dataset_handsigns = torchvision.datasets.ImageFolder("/content/HandSigns/resized_train_32", transform=train_transform_handsigns)
# テスト用のHandSignsデータセットの読み込み
test_dataset_handsigns = torchvision.datasets.ImageFolder("/content/HandSigns/resized_test_32", transform=test_transform_handsigns)

# 学習用のHandSignsデータローダーを作成
train_loader_handsigns = torch.utils.data.DataLoader(dataset=train_dataset_handsigns, batch_size=16, shuffle=True, num_workers=2)
# テスト用のHandSignsデータローダーを作成
test_loader_handsigns = torch.utils.data.DataLoader(dataset=test_dataset_handsigns, batch_size=16, shuffle=False, num_workers=2)

In [8]:
# 学習データのクラス名とラベルの対応を確認
print(train_dataset_handsigns.class_to_idx)
# テストデータのクラス名とラベルの対応を確認
print(test_dataset_handsigns.class_to_idx)

{'100000': 0, '100001': 1, '100111': 2, '101000': 3, '110000': 4, '110001': 5, '111000': 6, '111001': 7, '111100': 8, '111111': 9}
{'100000': 0, '100001': 1, '100111': 2, '101000': 3, '110000': 4, '110001': 5, '111000': 6, '111001': 7, '111100': 8, '111111': 9}


In [9]:
# イテレータを作成
data_iter = iter(train_loader_handsigns)

# データを1バッチ分取得
images, labels = next(data_iter)

# 取得したバッチの画像とラベルの情報を確認
print("画像の形状:", images.shape)  # (バッチサイズ, チャンネル数, 高さ, 幅)
print("ラベル:", labels)

画像の形状: torch.Size([16, 3, 32, 32])
ラベル: tensor([3, 7, 4, 1, 3, 5, 6, 9, 9, 5, 3, 0, 4, 0, 2, 1])


### モデルの実装

In [10]:
# 1×1の畳み込みを定義
def conv1x1(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False)

# 3×3の畳み込みを定義
def conv3x3(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)

In [11]:
# Residual Blocksを定義
class BuildingBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()
        self.bn1 = nn.BatchNorm2d(in_channels)
        self.conv1 = conv3x3(in_channels, out_channels, stride)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.conv2 = conv3x3(out_channels, out_channels)
        self.dropout = nn.Dropout(p=0.3)
        self.relu = nn.ReLU(inplace=True)

        # 入力と出力のチャンネル数が異なる場合（strideが1より大きい場合）、ダウンサンプリング
        if in_channels != out_channels or stride > 1:
            self.shortcut = conv1x1(in_channels, out_channels, stride)
        else:
            self.shortcut = nn.Sequential()

    def forward(self, x):
        out = self.bn1(x)
        out = self.relu(out)
        out = self.conv1(out)
        out = self.bn2(out)
        out = self.relu(out)
        out = self.dropout(out)
        out = self.conv2(out)
        # 残差写像と恒等写像の要素毎の和を計算
        out += self.shortcut(x)
        return out

In [12]:
class WideResNet(nn.Module):
    def __init__(self, block, depth, k, num_classes=10):
        super().__init__()
        assert (depth - 4) % 6 == 0, "depth should be 6n + 4"
        n = (depth - 4) // 6
        channels = [16, 16 * k, 32 * k, 64 * k]
        self.conv1 = conv3x3(3, channels[0])
        # Residual Blocks（1)
        self.layer1 = self._make_layer(block, channels[0], channels[1], n)
        # Residual Blocks（2）
        self.layer2 = self._make_layer(block, channels[1], channels[2], n, stride=2)
        # Residual Blocks（3）
        self.layer3 = self._make_layer(block, channels[2], channels[3], n, stride=2)
        self.bn = nn.BatchNorm2d(channels[3])
        self.relu = nn.ReLU(inplace=True)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(channels[3], num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                # Heの初期化（正規分布）
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                # 重みを1に初期化
                nn.init.constant_(m.weight, 1)
                # バイアスを0に初期化
                nn.init.constant_(m.bias, 0)

    # Residual Blocksを作成する関数を定義
    def _make_layer(self, block, in_channels, out_channels, blocks, stride=1):
        layers = []
        # 最初の Residual Block（stride=stride）
        layers.append(block(in_channels, out_channels, stride))
        # 残りの Residual Block（stride=1）
        for _ in range(1, blocks):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.bn(x)
        x = self.relu(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

### 学習と評価（50 epochs, learning_rate=0.001）

In [25]:
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義（一番精度の高かったモデルを採用）
model = torch.load('/content/drive/MyDrive/WideResNet28_10_CIFAR100_100epochs_CLRS_restest.pth')
# 最終層を変更
model.fc = nn.Linear(model.fc.in_features, 10)
# 最終層以外のパラメータの勾配計算を停止
for param in model.parameters():
    param.requires_grad = False
# 最終層のパラメータのみを学習するように設定
for param in model.fc.parameters():
    param.requires_grad = True
# モデルをデバイスに転送
model = model.to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 50
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_handsigns):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/WideResNet28_10_HandSigns_from_CIFAR100_100epochs_CLRS_50epochs_lr0.001_CLRS_restest.pth')

Epoch [1/50], Train Accuracy: 16.3 %, Loss: 2.1872
Epoch [1/50], Learning Rate: 0.0001
Epoch [2/50], Train Accuracy: 28.4 %, Loss: 1.9151
Epoch [2/50], Learning Rate: 0.00019
Epoch [3/50], Train Accuracy: 39.1 %, Loss: 2.1232
Epoch [3/50], Learning Rate: 0.00028
Epoch [4/50], Train Accuracy: 47.8 %, Loss: 1.7690
Epoch [4/50], Learning Rate: 0.00036999999999999994
Epoch [5/50], Train Accuracy: 51.5 %, Loss: 1.6091
Epoch [5/50], Learning Rate: 0.00045999999999999996
Epoch [6/50], Train Accuracy: 56.3 %, Loss: 1.4586
Epoch [6/50], Learning Rate: 0.00055
Epoch [7/50], Train Accuracy: 58.9 %, Loss: 1.2362
Epoch [7/50], Learning Rate: 0.0006399999999999999
Epoch [8/50], Train Accuracy: 63.1 %, Loss: 0.9741
Epoch [8/50], Learning Rate: 0.00073
Epoch [9/50], Train Accuracy: 63.5 %, Loss: 0.7284
Epoch [9/50], Learning Rate: 0.00082
Epoch [10/50], Train Accuracy: 68.9 %, Loss: 0.5754
Epoch [10/50], Learning Rate: 0.00091
Epoch [11/50], Train Accuracy: 66.4 %, Loss: 1.1605
Epoch [11/50], Learning

In [26]:
# 学習前後のモデルの最終層以外の重みを比較する関数を作成
def check_weight_change_except_last(model, model_init):
    for (name, param), (name_init, param_init) in zip(model.named_parameters(), model_init.named_parameters()):
        # 最終層を除く
        if name != 'fc.weight' and name != 'fc.bias':
            weights_changed = (param.data != param_init.data).any()
            if weights_changed:
                return print('最終層以外の重みが変化しています')
    return print('最終層以外の重みは変化していません')

# 学習前後のモデルの最終層の重みを比較する関数を作成
def check_weight_change_last(model, model_init):
    weights_changed = (model.fc.weight.data != model_init.fc.weight.data).any() or \
                      (model.fc.bias.data != model_init.fc.bias.data).any()
    if weights_changed:
        return print('最終層の重みは変化しています')
    else:
        return print('最終層の重みは変化していません')

# 学習前後でモデルの最終層以外の重みが変化していないかを確認
check_weight_change_except_last(model, model_init)
# 学習前後でモデルの最終層の重みが変化しているかを確認
check_weight_change_last(model, model_init)

最終層以外の重みは変化していません
最終層の重みは変化しています


In [27]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_handsigns:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 79.0 %


In [28]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### 学習と評価（50 epochs, learning_rate=0.005）

In [17]:
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義（一番精度の高かったモデルを採用）
model = torch.load('/content/drive/MyDrive/WideResNet28_10_CIFAR100_100epochs_CLRS_restest.pth')
# 最終層を変更
model.fc = nn.Linear(model.fc.in_features, 10)
# 最終層以外のパラメータの勾配計算を停止
for param in model.parameters():
    param.requires_grad = False
# 最終層のパラメータのみを学習するように設定
for param in model.fc.parameters():
    param.requires_grad = True
# モデルをデバイスに転送
model = model.to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.005
# 全体のepoch数を設定
num_epochs = 50
# warm-upするepoch数を設定
warmup_epochs = 5

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_handsigns):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/WideResNet28_10_HandSigns_from_CIFAR100_100epochs_CLRS_50epochs_lr0.005_CLRS_restest.pth')

Epoch [1/50], Train Accuracy: 17.8 %, Loss: 2.3667
Epoch [1/50], Learning Rate: 0.0001
Epoch [2/50], Train Accuracy: 25.3 %, Loss: 2.1458
Epoch [2/50], Learning Rate: 0.00108
Epoch [3/50], Train Accuracy: 40.8 %, Loss: 1.7269
Epoch [3/50], Learning Rate: 0.0020599999999999998
Epoch [4/50], Train Accuracy: 55.2 %, Loss: 1.2470
Epoch [4/50], Learning Rate: 0.0030399999999999997
Epoch [5/50], Train Accuracy: 56.9 %, Loss: 1.4606
Epoch [5/50], Learning Rate: 0.00402
Epoch [6/50], Train Accuracy: 60.2 %, Loss: 1.4957
Epoch [6/50], Learning Rate: 0.005
Epoch [7/50], Train Accuracy: 61.4 %, Loss: 1.7982
Epoch [7/50], Learning Rate: 0.004995165484649265
Epoch [8/50], Train Accuracy: 67.8 %, Loss: 0.8988
Epoch [8/50], Learning Rate: 0.004980681018220471
Epoch [9/50], Train Accuracy: 68.4 %, Loss: 1.0325
Epoch [9/50], Learning Rate: 0.004956603764285287
Epoch [10/50], Train Accuracy: 69.5 %, Loss: 1.1446
Epoch [10/50], Learning Rate: 0.0049230287447651466
Epoch [11/50], Train Accuracy: 67.5 %, L

In [18]:
# 学習前後のモデルの最終層以外の重みを比較する関数を作成
def check_weight_change_except_last(model, model_init):
    for (name, param), (name_init, param_init) in zip(model.named_parameters(), model_init.named_parameters()):
        # 最終層を除く
        if name != 'fc.weight' and name != 'fc.bias':
            weights_changed = (param.data != param_init.data).any()
            if weights_changed:
                return print('最終層以外の重みが変化しています')
    return print('最終層以外の重みは変化していません')

# 学習前後のモデルの最終層の重みを比較する関数を作成
def check_weight_change_last(model, model_init):
    weights_changed = (model.fc.weight.data != model_init.fc.weight.data).any() or \
                      (model.fc.bias.data != model_init.fc.bias.data).any()
    if weights_changed:
        return print('最終層の重みは変化しています')
    else:
        return print('最終層の重みは変化していません')

# 学習前後でモデルの最終層以外の重みが変化していないかを確認
check_weight_change_except_last(model, model_init)
# 学習前後でモデルの最終層の重みが変化しているかを確認
check_weight_change_last(model, model_init)

最終層以外の重みは変化していません
最終層の重みは変化しています


In [19]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_handsigns:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 82.0 %


In [20]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### 学習と評価（50 epochs, learning_rate=0.01）

In [21]:
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義（一番精度の高かったモデルを採用）
model = torch.load('/content/drive/MyDrive/WideResNet28_10_CIFAR100_100epochs_CLRS_restest.pth')
# 最終層を変更
model.fc = nn.Linear(model.fc.in_features, 10)
# 最終層以外のパラメータの勾配計算を停止
for param in model.parameters():
    param.requires_grad = False
# 最終層のパラメータのみを学習するように設定
for param in model.fc.parameters():
    param.requires_grad = True
# モデルをデバイスに転送
model = model.to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.01
# 全体のepoch数を設定
num_epochs = 50
# warm-upするepoch数を設定
warmup_epochs = 5

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_handsigns):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/WideResNet28_10_HandSigns_from_CIFAR100_100epochs_CLRS_50epochs_lr0.01_CLRS_restest.pth')

Epoch [1/50], Train Accuracy: 14.9 %, Loss: 2.0921
Epoch [1/50], Learning Rate: 0.0001
Epoch [2/50], Train Accuracy: 26.7 %, Loss: 2.0453
Epoch [2/50], Learning Rate: 0.00208
Epoch [3/50], Train Accuracy: 46.4 %, Loss: 1.0604
Epoch [3/50], Learning Rate: 0.00406
Epoch [4/50], Train Accuracy: 57.6 %, Loss: 1.5830
Epoch [4/50], Learning Rate: 0.00604
Epoch [5/50], Train Accuracy: 60.5 %, Loss: 0.6368
Epoch [5/50], Learning Rate: 0.00802
Epoch [6/50], Train Accuracy: 61.2 %, Loss: 0.6475
Epoch [6/50], Learning Rate: 0.01
Epoch [7/50], Train Accuracy: 60.5 %, Loss: 2.6591
Epoch [7/50], Learning Rate: 0.009990232305719944
Epoch [8/50], Train Accuracy: 64.1 %, Loss: 3.0240
Epoch [8/50], Learning Rate: 0.009960967771506667
Epoch [9/50], Train Accuracy: 65.9 %, Loss: 2.5505
Epoch [9/50], Learning Rate: 0.00991232189110701
Epoch [10/50], Train Accuracy: 65.4 %, Loss: 1.5400
Epoch [10/50], Learning Rate: 0.009844486647586723
Epoch [11/50], Train Accuracy: 65.9 %, Loss: 3.9026
Epoch [11/50], Lear

In [22]:
# 学習前後のモデルの最終層以外の重みを比較する関数を作成
def check_weight_change_except_last(model, model_init):
    for (name, param), (name_init, param_init) in zip(model.named_parameters(), model_init.named_parameters()):
        # 最終層を除く
        if name != 'fc.weight' and name != 'fc.bias':
            weights_changed = (param.data != param_init.data).any()
            if weights_changed:
                return print('最終層以外の重みが変化しています')
    return print('最終層以外の重みは変化していません')

# 学習前後のモデルの最終層の重みを比較する関数を作成
def check_weight_change_last(model, model_init):
    weights_changed = (model.fc.weight.data != model_init.fc.weight.data).any() or \
                      (model.fc.bias.data != model_init.fc.bias.data).any()
    if weights_changed:
        return print('最終層の重みは変化しています')
    else:
        return print('最終層の重みは変化していません')

# 学習前後でモデルの最終層以外の重みが変化していないかを確認
check_weight_change_except_last(model, model_init)
# 学習前後でモデルの最終層の重みが変化しているかを確認
check_weight_change_last(model, model_init)

最終層以外の重みは変化していません
最終層の重みは変化しています


In [23]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_handsigns:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 89.0 %


In [24]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()